In [1]:
import numpy as np
import zlib
from PIL import Image
import base64
import cv2
import io
import json
import os
from Mask2Base64 import base64_2_mask

# def base64_2_mask(s):
#     z = zlib.decompress(base64.b64decode(s))
#     n = np.frombuffer(z, np.uint8)
#     mask = cv2.imdecode(n, cv2.IMREAD_UNCHANGED)[:, :, 3].astype(bool)
#     return mask

DATA_FOLDER = 'data_all' # <-- may have a different name
# assuming this ipynb resides in a folder inside project root directory
DATA_PATH = os.path.join(os.path.dirname(os.path.realpath("__file__")), os.pardir, DATA_FOLDER)

ANN_FOLDER = 'ann'
IMG_FOLDER = 'img'
LABEL_FOLDER = 'labels' # <-- this folder should be created beforehand

def create_txt_file(labels_path: str, file_prefix: str):
    json_path = os.path.join(DATA_PATH, ANN_FOLDER, file_prefix+'.png.json')
    with open(json_path) as f:
        file_content = f.read()
        annotation = json.loads(file_content)
    img_height = annotation['size']['height']
    img_width = annotation['size']['width']
    with open(os.path.join(labels_path, file_prefix+'.txt'), 'w') as f:
        for i, obj in enumerate(annotation['objects']):
            left_corner_x, left_corner_y = [int(s) for s in annotation['objects'][i]['bitmap']['origin']]
            bbox_height, bbox_width = base64_2_mask(annotation['objects'][i]['bitmap']['data']).shape
            x_center = left_corner_x + bbox_width // 2
            y_center = left_corner_y + bbox_height // 2
            output_arr = [0, x_center / img_width, y_center / img_height, bbox_width / img_width, bbox_height / img_height]
            output_str = ' '.join([str(s) for s in output_arr])
            f.write(output_str + '\n')

In [2]:
create_txt_file(os.path.join(DATA_PATH, LABEL_FOLDER), 'IMG_1749')

In [3]:
for file_prefix in list(name.replace('.png', '') for name in os.listdir(os.path.join(DATA_PATH, IMG_FOLDER))):
    create_txt_file(os.path.join(DATA_PATH, LABEL_FOLDER), file_prefix)